About this script:

Created by: Andrew Parkin with a lot of help and support from Phil Baranyai 

Work might have prevented you from helping me finish it but you're still #SpatialAF

This script will find items in your enviroment of choice (AGOL or Enterpris/Portal) and see how they relate to one another. 

If you want to see how your items relate to one another through your entire online envrironment see the Dependencies Dual Environment notebook

In [ ]:
from arcgis.gis import GIS
import pandas as pd
import os

In [ ]:
# Setup Date/time variables
date = datetime.date.today().strftime("%Y%m%d")
Time = time.strftime("%H%M", time.localtime())

The cell below is the only spot that requires user input once inputed the script will run as is just need to check back in on it as it runs.

In [ ]:
site = input('Please copy and paste your Agol or Portal website here: ')
username = input('Please copy and paste your Username here: ')
password = input('Please copy and paste your Password here: ')

print('Connecting to your enviroment..')
connection = GIS(site, username, password)
print('Connected to your enviroment..')

In [ ]:
# Setup export path to user's documents folder
userprofile = os.environ['USERPROFILE']
ReportDirectory = userprofile+"\\Documents\\WebGISDependenciesReports"
reportdirExists = os.path.exists(ReportDirectory)
if not reportdirExists:
    os.makedirs(ReportDirectory)
    print(ReportDirectory+" was not found, so it was created")
ExcelOutput = os.path.join(ReportDirectory,'Dependencies_report_'+str(date)+'.xlsx')

In [ ]:
# Variables
ItemList = ['Administrative Report', 'Apache Parquet', 'CAD Drawing', 'CSV', 'Color Set', 'Content Category Set',
            'Document Link', 'Earth Configuration', 'Esri Classifier Definition', 'Export Package',
            'Feature Collection', 'Feature Collection Template', 'Feature Service', 'File Geodatabase', 'GeoJson',
            'GML', 'GeoPackage', 'Geocoding Service', 'Geodata Service', 'Geometry Service', 'Geoprocessing Service',
            'Globe Service', 'Image', 'KML', 'KML CollectionMap Service', 'Microsoft Excel', 'Microsoft Powerpoint',
            'Microsoft Word', 'Network Analysis Service', 'OGCFeatureServer', 'Oriented Imagery Catalog', 'PDF',
            'Relational Database Connection', 'Relational Database Connection', 'Report Template', 'SQLite Geodatabase',
            'Scene Service', 'Service Definition', 'Shapefile', 'Statistical Data Collection', 'StoryMap Theme',
            'Style', 'Symbol SetImage Service', 'Vector Tile Service', 'Visio Document', 'WFS', 'WMS', 'WMTS',
            'Workflow Manager Service', 'iWork Keynote', 'iWork Numbers', 'iWork Pages']

AppList = ['360 VR Experience', 'AppBuilder Extension', 'AppBuilder Widget Package', 'CityEngine Web Scene',
           'Code Attachment', 'Dashboard', 'Deep Learning Studio Project', 'Esri Classification Schema',
           'Excalibur Imagery Project', 'Experience Builder Widget', 'Experience Builder Widget Package', 'Form',
           'GeoBIM Application', 'GeoBIM Project', 'Hub Event', 'Hub Initiative', 'Hub Initiative Template', 'Hub Page',
           'Hub Project', 'Hub Site Application', 'Insights Data Engineering Model',
           'Insights Data Engineering Workbook', 'Insights Model', 'Insights Page', 'Insights Theme',
           'Insights Workbook', 'Insights Workbook Package', 'Investigation', 'Mission', 'Mobile Application',
           'Native Application', 'Native Application Installer', 'Notebook', 'Notebook Code Snippet Library',
           'Operation View', 'Operations Dashboard Add In', 'Operations Dashboard Extension', 'Ortho Mapping Project',
           'Ortho Mapping Template', 'Pro Map', 'StoryMap', 'Web AppBuilder WidgetSolution', 'Web Experience',
           'Web Experience Template', 'Web Map', 'Web Mapping Application', 'Web Scene', 'Workforce Project']

Items = []
Apps = []

# List to store all my Function Results
FuncResults = []

In [ ]:
# ***Defining function that will gather the information we need. ***
def my_func(connection, item, webapps):
    # For some reason script won't pull in certain Urls so try and except added to prevent script breaking.
    # If url cant be pulled it will still collect as much as the item info added.
    try:
        # Item info
        item_info = item
        find_id = item_info.id
        find_url = connection.content.get(find_id).url
        item_title = item_info.title
        item_owner = item_info.owner
        item_type = item_info.type
        item_create_date = datetime.datetime.fromtimestamp(round(item_info.created / 1000))
        item_modified_date = datetime.datetime.fromtimestamp(round(item_info.modified / 1000))
        item_size = item_info.size/1024

        # Collects Web maps then Return subset of map IDs which contain the service URL we're looking for
        webmaps = connection.content.search('', item_type='Web Map', max_items=-1)
        matches = [m.id for m in webmaps if str(m.get_data()).find(find_url) > -1]

        # Create empty list to populate with results
        app_list = []

        # Check each web app for matches
        for w in Apps:

            try:
                # Get the JSON as a string
                wdata = str(w.get_data())

                criteria = [
                    wdata.find(find_url) > -1,  # Check if URL is directly referenced
                    any([wdata.find(m) > -1 for m in matches])  # Check if any matching maps are in app
                ]

                # If layer is referenced directly or indirectly, append app to list
                if any(criteria):
                    app_list.append(w)

            # Some apps don't have data, so we'll just skip them if they throw a TypeError
            except:
                continue

        if len(app_list) > 0:
            for a in app_list:
                FuncResults.append({'Environment': connection_type, 'Item Name': item_title, 'Item Type': item_type,
                                    'Item ID': find_id, 'Item Url': find_url, 'Item Owner': item_owner,
                                    'Item Create Date': item_create_date, 'Item Modified Date': item_modified_date,
                                    'Item Size (KB)': item_size, 'Application Name': a.title,
                                    'Application Type': a.type, 'Application ID': a.id, 'Application Owner': a.owner,
                                    'Application Create Date': datetime.datetime.fromtimestamp(round(a.created / 1000)),
                                    'Application Modified Date': datetime.datetime.fromtimestamp(round(a.modified / 1000)),
                                    'Application Size (KB)': a.size/1024})

        if len(app_list) == 0:
            FuncResults.append({'Environment': connection_type, 'Item Name': item_title, 'Item Type': item_type,
                                'Item ID': find_id, 'Item Url': find_url, 'Item Owner': item_owner,
                                'Item Create Date': item_create_date, 'Item Modified Date': item_modified_date,
                                'Item Size (KB)': item_size, 'Application Name': 'N/A', 'Application Type': 'N/A',
                                'Application ID': 'N/A', 'Application Owner': 'N/A', 'Application Create Date': 'N/A',
                                'Application Modified Date': 'N/A', 'Application Size (KB)': 'N/A'})


    except:
        print("Exception found gathering baseline item info.")
        item_info = item
        find_id = item_info.id
        find_url = connection.content.get(find_id).url
        item_title = item_info.title
        item_owner = item_info.owner
        item_type = item_info.type

        FuncResults.append({'Environment': connection_type, 'Item Name': item_title, 'Item Type': item_type,
                            'Item ID': find_id, 'Item Url': 'Cant find Url', 'Item Owner': item_owner,
                            'Item Create Date': item_create_date, 'Item Modified Date': item_modified_date,
                            'Item Size (KB)': item_size, 'Application Name': 'N/A', 'Application Type': 'N/A',
                            'Application ID': 'N/A', 'Application Owner': 'N/A', 'Application Create Date': 'N/A',
                            'Application Modified Date': 'N/A', 'Application Size (KB)': 'N/A'})


In [ ]:
# Combining all Item types into a single list
print('Collecting Item Data!')
for i in ItemList:
    Temp = connection.content.search('', item_type=i, max_items=-1)
    
    if len(Temp) > 0:
        for t in Temp:
            Items.append(t)

print('You have ' + str(len(Items)) + 'items in your enviroment')
print('Collected Item Data!')

In [ ]:
print('Collecting Application Data..')

# Combining all Apps into a single list

for al in AppList:
    Temp = connection.content.search('', item_type=al, max_items=-1)
    if len(Temp) > 0:
        for t in Temp:
            Apps.append(t)
            
print('You have ' + str(len(Apps)) + 'apps in your enviroment')
print('Collected Application Data!')

In [ ]:
print('Starting Items')
for i in Items:
    print('working on ' + str(i.title))
    my_func(connection, i, Apps)
print('Finished Items')

In [ ]:
# Creating dataframe
print('Creating Data Frame')

df = pd.DataFrame(FuncResults)

# Sorting by Url with Portal items on top of AGOL
df1 = df.sort_values(by=['Item Url'], ascending=[True])
# Reindexing
df2 = df1.reset_index(drop=True)

# Creating Mask that will put a line of separation between each Item
mask = df2['Item Url'].ne(df2['Item Url'].shift(-1))
df3 = pd.DataFrame('', index=mask.index[mask] + .5, columns=df.columns)

# Creating the Final Dataframe.
new_df = pd.concat([df2, df3]).sort_index().reset_index(drop=True).iloc[:-1]
print('Finished Data Frame')
# Exporting Dataframe to excel
print('Exporting to Excel')
new_df.to_excel(ExcelOutput, index=False)
print('Finished')